In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import numpy
import sympy
from sympy.solvers import solve
from sympy import diff
from time import sleep
from progressbar import progressbar
from math import e as euler
from math import log as ln

MAX_ITERATION = 500
MAX_ITERATION_QUASE_NEWTON = 5000
MAX_ITERATIONITERATION_DICOTOMICA = 5000

In [5]:
def function(x):
    return pow(x[0]-2, 4) + pow(x[0]-2*x[1], 2)

In [6]:
def grad_function(x, f, c):
    if f == 1:
        return [(4*pow(x[0]-2, 3) + 2*(x[0]-2*x[1]) + ((2*c*x[0])/pow(x[0]*x[0]-x[1], 2))), 
                (-c/pow(x[0]*x[0] - x[1], 2))  -4*(x[0]-2*x[1])]
    if f == 2:
        p_a = -4*pow(x[0], 5)
        p_b = 24*pow(x[0], 4)
        p_c = 4*x[0]*x[0]*x[0]*x[1]
        p_d = -50*pow(x[0], 3)
        p_e = -20*x[0]*x[0]*x[1]
        p_f = 50*x[0]*x[1]
        p_g = 2*c*x[0]
        p_h = -4*x[1]*x[1]
        p_i = x[1]-x[0]*x[0]
        p_p = ((p_a + p_b + p_c + p_d  +p_e + p_f + p_g + p_h) / p_i) - 32
        return [p_p, 8*x[1]-(c/(x[1]-x[0]*x[0]))-4*x[0]]

In [7]:
#Passo 4
def function_phi(x, f, c):
    if f == 1:
        c_b = -c / (x[0]*x[0]-x[1])
    if f == 2: #(barreira de Frisch)
        temp = x[0]*x[0]-x[1]
        if temp > 0:
            temp = -temp
        c_b = -c * ln(-(temp))
    phi = function(x) + c_b
    return phi

In [8]:
def busca_dicotomica(a_min, a_max, e, f, x_i, d_i, c):
    for k in range(MAX_ITERATIONITERATION_DICOTOMICA):
        if abs(a_max/a_min) < 1 + e*10 and k > 5:
                break
                
        lambda_k = ((a_min+a_max)/2)-e
        lambda_x = [x_i[j] + d_i[j] * lambda_k for j in range(len(x_i))]
        f_lambda = function_phi(lambda_x, f, c)
        
        mi_k = ((a_min+a_max)/2)+e
        mi_x = [x_i[j] + d_i[j] * mi_k for j in range(len(x_i))]
        f_mi = function_phi(mi_x, f, c)
            
        if f_lambda < f_mi:
            a_max = mi_k
        else:
            a_min = lambda_k
    return (a_min+a_max)/2

In [9]:
def DFP(input_D_i, input_s_i, input_y_i):
    s_i = numpy.array([[i] for i in input_s_i])
    y_i = numpy.array([[i] for i in input_y_i])
    D_i = numpy.array(input_D_i)
    p1 = numpy.true_divide(s_i.dot(s_i.transpose()), s_i.transpose().dot(y_i))
    div_p2 = y_i.transpose().dot(D_i).dot(y_i)
    nom_p2 = D_i.dot(y_i).dot(y_i.transpose()).dot(D_i)
    p2 = numpy.true_divide(nom_p2, div_p2)
    return D_i + p1 - p2

In [10]:
def metodo_quase_newton(x_i, e, f, c):
    #Passo 3
    D_i = numpy.identity(len(x_i))
    for i in range(MAX_ITERATION_QUASE_NEWTON):
        
        #Passo 1
        gradiente = grad_function(x_i, f, c)
        #print(f"gradiente = {gradiente}")
        
        #Passo 4
        d_i = -D_i.dot(gradiente)
        #print(f"d_i = {d_i}")
        
        #Passo 5
        alpha = busca_dicotomica(0.1, 1, e/100, f, x_i, d_i, c)
        #print(f"alpha = {alpha}")

        #Passo 6
        antigo_x_i = x_i
        x_i = [x_i[k] + alpha*d_i[k] for k in range(len(x_i))]
        #print(f"i = {i}, x_i = {x_i}, alpha = {alpha}, d_i = {d_i}")
        
        antigo_gradiente = gradiente
        gradiente = grad_function(x_i, f, c)
        #print(f"novo_gradiente = {gradiente}")
        
        #Passo 7
        convergiu = True
        for k in range(len(x_i)):
            if abs(antigo_x_i[k] - x_i[k]) > e:
                convergiu = False
        if convergiu:
            break
            
        #Passo 8
        s_i = [x_i[k] - antigo_x_i[k] for k in range(len(x_i))]
        y_i = [gradiente[k] - antigo_gradiente[k] for k in range(len(gradiente))]
        #print(f"s_i = {s_i}")
        #print(f"y_i = {y_i}")
        
        #Passo 9
        np_s_i = numpy.array(s_i)
        np_y_i = numpy.array(y_i)
        result = numpy.dot(np_s_i.transpose(), np_y_i)
        
        #Passo 9
        if result < 0:
            D_i = numpy.identity(len(x_i))
        
        #Passo 10
        if result >= 0:
            D_i = DFP(D_i, np_s_i, np_y_i)
        
        #print(f"D_i+1 = {D_i}")
        #print(f"result = {result}") 
    #print(x_i)
    return x_i

In [11]:
def metodo_das_barreiras(x_i, e, f, c_0, inc_c_0):
    for i in progressbar(range(MAX_ITERATION)):
        #Passo 5
        antigo_x_i = x_i
        x_i = metodo_quase_newton(x_i, e, f, c_0)
            
        #Passo 6 
        convergiu = True
        for k in range(len(x_i)):
            if abs(antigo_x_i[k] - x_i[k]) > e/100:
                convergiu = False
        if convergiu:
            print(f"Numero de iterações foi o suficiente para convergir")
            break
            
        #Passo 7
        c_0 = c_0/inc_c_0
    return x_i

In [14]:
x, y = sympy.symbols("x y")
f1 = (x-2)**4 + (x-2*y)**2

mdb = metodo_das_barreiras([1, 2], 0.0001, 1, 10, 10)
sp = solve([f1.diff(x), f1.diff(y)], [x, y])
print(f"\nFunção 1 - Valor cálculado pelo método das penalidades (com barreira clássica): {mdb}\nValor otimizado : {sp}")

100% (500 of 500) |######################| Elapsed Time: 0:00:06 Time:  0:00:06



Função 1 - Valor cálculado pelo método das penalidades (com barreira clássica): [1.9810076535327212, 0.9905052633063269]
Valor otimizado : [(2, 1)]


In [13]:
mdb = metodo_das_barreiras([1, 2], 0.0001, 2, 10, 10)
print(f"\nFunção 1 - Valor cálculado pelo método das penalidades (com barreira Frisch): {mdb}\nValor otimizado : {sp}")

100% (500 of 500) |######################| Elapsed Time: 0:00:07 Time:  0:00:07



Função 1 - Valor cálculado pelo método das penalidades (com barreira Frisch): [2.0242513076343753, 1.0121227446005903]
Valor otimizado : []
